In [ ]:
import pandas as pd
from simple_salesforce import Salesforce, SalesforceLogin
import pyodbc, sys, time 
import requests
import numpy as np
from datetime import datetime

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 


Source_data = pd.read_csv("/mnt/c/Users/ralvin/Downloads/837I00111182024.csv")



Provider_TIN_dict = {'MAPPINGS':{'TIN':['Name', 'Unique_TIN__c'], 'Address':['Address__c'], 'City':['City__c'], 'State':['State__c'], 'Zip':['Zip__c'],
                      'Provider':['Provider_Name__c']}, 'KEYS':{'Provider_TIN__c':'Unique_TIN__c'}}

Account_dict = {'MAPPINGS':{'TIN':['Tax_ID__c'], 'BillProvAddress1':['BillingAddress'], 'BillProvCity':['BillingCity'], 'BillProvState':['BillingState'],
                 'BillProvZip':['BillingPostalCode'], 'Provider':['Name']}, 'KEYS':{'Account':'Tax_ID__c'}}



Claim_dict = {'MAPPINGS':{'Claim ID':['Claim_ID__c'], 'TIN':['Provider_TIN__c'], 'Patient Group/Policy Number':['Group_Client__c'], 'Claim Number':['Claim_Number__c'],'JurisdictionState':['Jurisdiction__c'],
               'Diag Code 1st':['Diag_Code__c'],'Diag Code 2nd':['Diag_Code_2nd__c'], 'Diag Code 3rd':['Diag_Code_3rd__c'], 'Diag Code 4th':['Diag_Code_4th__c'], 'Provider':['Provider__c'],
               'Patient':['Patient__c'], 'Patient ID':['Patient_ID__c'], 'Date of Birth':['Date_of_Birth__c'],'Zip':['Provider_Zip__c'], 'DRG':['DRG__c'], 'IsQPA':['QPA__c'], 'NPI':['NPI__c'],
                'PlaceofService':['POS__c'], 'Billing Provider Taxonomy':['Provider_Specialty__c', 'Service_Type__c']}, 'KEYS':{'Claims__c':'Claim_ID__c'}, 
               'TABLE_CONVERT':{'TIN':{'Provider_TIN__c':['Name', 'Id']}, 'Patient Group/Policy Number':{'Groups_Clients__c':['Group_Number__c', 'Id']}, 'JurisdictionState':{'Jurisdiction__c':['Name', 'Id']},
                                 'DRG':{'DRG__c':['Name', 'Id']}, 'Billing Provider Taxonomy':{'Provider_Specialty__c':['Name', 'Id', 'Service_Type__c']}},
                'KEYS':{'Claims__c':'Claim_ID__c'}
               }

Line_item_dict = {'MAPPINGS':{'Begin DOS':['DOS__c'], 'Revenue Code':['Rev_Code__c'],'Billed Amount':['Billed_Amount__c'], 'Claim ID':['RHP_ID__c'], 'Mod':['Mod__c'],
                   'Mod (2nd)':['Mod_2nd__c'], 'Units':['Units__c'], 'MAR':['MAR__c'], 'HCPCS/CPT Code':['HCPCS_CPT_Code__c'], 'Line ID':['Line_ID__c'], 
                    '4LCodes':['Exp_Code_2nd__c']},
                  'KEYS':{'Line_Items__c':'RHP_ID__c'},
                  'TABLE_CONVERT':{'Claim ID':{'Claims__c':['Claim_ID__c']}, 'HCPCS/CPT Code':{'HCPCS_CPT_Code__c':['Name', 'Id']}}
                  }





def salesforce_connection(sandbox=True):
    session = requests.Session()
    security_token = 'pIjwc8P0OB2ZisbQYhlKxfLd'
    username = 'ralvin@relianthp.com.tctdev'
    password = 'Flash363!'
    domain = 'test'

    session_id, instance = SalesforceLogin(username=username, password=password, security_token=security_token, domain=domain)
    sf = Salesforce(instance=instance, session_id=session_id, session=session)
    print('Connected to Salesforce (Sandbox)')
    return sf

sf = salesforce_connection()

all_objects = sf.describe()["sobjects"]


accounts_metadata = sf.Account.describe()
account_cols = [field['name'] for field in accounts_metadata['fields']]

provider_tin_metadata = sf.Provider_TIN__c.describe()
provider_tin_cols = [field['name'] for field in provider_tin_metadata['fields']]

claims_metadata = sf.Claims__c.describe()
claim_cols = [field['name'] for field in claims_metadata['fields']]

line_item_metadata = sf.Line_Items__c.describe()
line_item_cols = [field['name'] for field in line_item_metadata['fields']]

clients_metadata = sf.Clients__c.describe()
clients_cols = [field['name'] for field in clients_metadata['fields']]

group_clients_metadata = sf.Groups_Clients__c.describe()
group_clients_cols = [field['name'] for field in group_clients_metadata['fields']]

jurisdiction_metadata = sf.Jurisdiction__c.describe()
jurisdiction_cols = [field['name'] for field in jurisdiction_metadata['fields']]

drg_metadata = sf.DRG__c.describe()
drg_cols = [field['name'] for field in drg_metadata['fields']]

hcpcs_cpt_metadata = sf.HCPCS_CPT_Code__c.describe()
hcpcs_cpt_cols = [field['name'] for field in hcpcs_cpt_metadata['fields']]

provider_specialty_metadata = sf.Provider_Specialty__c.describe()
provider_specialty_cols = [field['name'] for field in provider_specialty_metadata['fields']]

needed_objects = { "Claims__c": claim_cols, "Line_Items__c": line_item_cols, "Provider_TIN__c": provider_tin_cols, 
                 'Jurisdiction__c': jurisdiction_cols, 'Groups_Clients__c': group_clients_cols, 'DRG__c': drg_cols, 
                 'HCPCS_CPT_Code__c': hcpcs_cpt_cols, 'Account':account_cols, 'Clients__c': clients_cols, 'Provider_Specialty__c':provider_specialty_cols}


DATA_MAPPINGS = {'Provider_Object_Map':Provider_TIN_dict, 'Account_Object_Map':Account_dict, 'Claim_Object_Map':Claim_dict, 'Line_Item_Object_Map':Line_item_dict}




table_store = {}

for object_name in needed_objects:
    print(object_name)

    object_fields = needed_objects[object_name]

    fields_str = ", ".join(object_fields)
    query = f"SELECT {fields_str} FROM {object_name}"
    results = sf.query_all(query)
    records = results['records']
    df = pd.DataFrame(records).drop(columns='attributes')
    print(df.shape)
    table_store[object_name] = df

Connected to Salesforce (Sandbox)
Claims__c
(13033, 100)
Line_Items__c
(22789, 48)
Provider_TIN__c
(10853, 27)
Jurisdiction__c
(53, 16)
Groups_Clients__c
(1702, 58)
DRG__c
(808, 20)
HCPCS_CPT_Code__c
(10045, 36)
Account
(2217, 98)
Clients__c
(62, 58)
Provider_Specialty__c
(870, 18)


In [74]:

# table_store['Line_Items__c'].head()

In [61]:
# foreign_table = list(map_table.values())[0]
# foreign_table

In [60]:

# Line_item_dict = {'MAPPINGS':{'Begin DOS':['DOS__c'], 'Revenue Code':['Rev_Code__c'],'Billed Amount':['Billed_Amount__c'], 'Claim ID':['RHP_ID__c'], 'Mod':['Mod__c'],
#                    'Mod (2nd)':['Mod_2nd__c'], 'Units':['Units__c'], 'MAR':['MAR__c'], 'HCPCS/CPT Code':['HCPCS_CPT_Code__c'], 'Line ID':['Line_ID__c'], 
#                     '4LCodes':['Exp_Code_2nd__c']},
#                   'KEYS':{'Line_Items__c':'RHP_ID__c'},
#                   'TABLE_CONVERT':{'Claim ID':{'Claims__c':['Claim_ID__c']}, 'HCPCS/CPT Code':{'HCPCS_CPT_Code__c':['Name', 'Id']}}
#                   }

# for source_col, target_cols in Line_item_dict['MAPPINGS'].items():
#     print(source_col)
#     print(target_cols)
#     print('')

In [71]:
def create_df(source_dict, Source_data, table_store):
    new_columns = [col for mapped_cols in source_dict['MAPPINGS'].values() for col in mapped_cols]
    new_df = pd.DataFrame(columns=new_columns)

    try:
        Table_convert_cols = list(source_dict['TABLE_CONVERT'].keys())
    except:
        Table_convert_cols = []

    print(Table_convert_cols)

    for source_col, target_cols in source_dict['MAPPINGS'].items():
        if source_col in Table_convert_cols:
            map_table = source_dict['TABLE_CONVERT'][source_col]
            print(map_table)
            
            try:
                string_ids = [
                        str(int(float(id))) if pd.notna(id) and id != 'nan' else np.nan
                        for id in Source_data[source_col]                               #### handeling DRG col
                    ]
            except:
                string_ids = [str(id) for id in Source_data[source_col]]
         
            Source_data[source_col] = string_ids #### col update to match same type in foreign table
            foreign_table = list(map_table.keys())[0]
            print(foreign_table)

            merged = Source_data.merge(table_store[foreign_table].loc[:, map_table[foreign_table]], left_on=source_col, right_on=map_table[foreign_table][0], how='left')

            for idx, col in enumerate(map_table[foreign_table], start=0):  # Start from the first column
                if len(map_table[foreign_table]) == 1:
                    # Handle the edge case where only one column exists
                    target_col = target_cols[0]  # Use the first target column
                    print(f"Single column mapping: {col} to {target_col}")
                    new_df[target_col] = merged[col]
                    break  # Exit the loop since there is only one column to map
                elif idx > 0 and idx - 1 < len(target_cols):  # For columns after the first
                    target_col = target_cols[idx - 1]  # Adjust indexing to match target columns
                    print(f"Mapping {col} to {target_col}")
                    new_df[target_col] = merged[col]
                else:
                    print(f"Skipping column {col} or no target column available.")


        if source_col not in Table_convert_cols:
            if source_col in Source_data.columns:
                for target_col in target_cols:
                    new_df[target_col] = Source_data[source_col]
        

    keys = source_dict['KEYS']


    return new_df, keys




def updated_data_pull(object_name, needed_objects):

    object_fields = needed_objects[object_name]
    fields_str = ", ".join(object_fields)
    query = f"SELECT {fields_str} FROM {object_name}"
    results = sf.query_all(query)

    records = results['records']
    df = pd.DataFrame(records).drop(columns='attributes')  # Remove unwanted attributes column
    
    return df




def format_date(value):
    try:
        # Attempt to parse and format the date
        return datetime.strptime(value, '%m/%d/%Y').strftime('%Y-%m-%d')
    except (ValueError, TypeError):
        # Return the value as-is if it’s not a valid date
        return value



def upsert(new_df, keys):


    errors = []
    object_name = list(keys.keys())[0]
    sf_object = getattr(sf, object_name)
    object_key = list(keys.values())[0]

    # print(object_key)

    for i in range(len(new_df)):
        # print(i)
        record = new_df.iloc[i, :].to_dict()
        # print(record)
        record_upsert = {k: v for k, v in record.items() if k != object_key}
        # Replace NaN with None
        record_upsert = {k: (None if pd.isna(v) else v) for k, v in record_upsert.items()}
        # Format date fields
        record_upsert = {k: format_date(v) if 'Date' in k or 'DOS' in k else v for k, v in record_upsert.items()}
        # fix Npi max length error
        record_upsert = {k: (str(v).split('.')[0][:10] if k == 'NPI__c' and v is not None else v) for k, v in record_upsert.items()}



    
        print(record_upsert)
        
        try:
            sf_object.upsert(f"{object_key}/{record[object_key]}", record_upsert)
            # pass
        except Exception as e:
            print(f"Error during upsert: {e}")
            errors.append(e)

    return errors

In [45]:
Lines_map = DATA_MAPPINGS['Line_Item_Object_Map']

claims_table_repulled = updated_data_pull('Claims__c', needed_objects)
### create lines table
map_lines_table, keys = create_df(Lines_map, Source_data, table_store)

['Claim ID', 'HCPCS/CPT Code']
{'Claims__c': ['Claim_ID__c']}
Claims__c
Single column mapping: Claim_ID__c to RHP_ID__c
{'HCPCS_CPT_Code__c': ['Name', 'Id']}
HCPCS_CPT_Code__c
Skipping column Name or no target column available.
Mapping Id to HCPCS_CPT_Code__c


In [75]:
# map_lines_table.head()

In [ ]:
Povider_map = DATA_MAPPINGS['Provider_Object_Map']
Account_map = DATA_MAPPINGS["Account_Object_Map"]
Claims_map = DATA_MAPPINGS['Claim_Object_Map']
Lines_map = DATA_MAPPINGS['Line_Item_Object_Map']

### First step - populate providers table
map_account_table, _ = create_df(Account_map, Source_data, table_store)
### Second step - populate providers table
map_provider_table, keys = create_df(Povider_map, Source_data, table_store)
### Third step = Upsert data onto SF
update_provider_object = upsert(map_provider_table, keys)
### Fourth step - repull provider table to update claims table
provider_table_repulled = updated_data_pull('Provider_TIN__c', needed_objects)
### Fifth step - Populate Claims table
map_claims_table, keys = create_df(Claims_map, Source_data, table_store)
### Upsert Claims data to sf 
update_claims_object = upsert(map_claims_table, keys)
### pull updated claims
claims_table_repulled = updated_data_pull('Claims__c', needed_objects)
### create lines table
map_lines_table, keys = create_df(Lines_map, Source_data, table_store)
### upsert lines data
update_lines_object = upsert(map_lines_table, keys)












[]
[]
{'Name': 941105628, 'Address__c': 'FILE 54602', 'City__c': 'LOS ANGELES', 'State__c': 'CA', 'Zip__c': 900749998, 'Provider_Name__c': 'KAISER FOUNDATION HOSPITALS '}
{'Name': 351166081, 'Address__c': '1125 W JEFFERSON STREET', 'City__c': 'FRANKLIN', 'State__c': 'IN', 'Zip__c': 461312140, 'Provider_Name__c': 'JOHNSON MEMORIAL HOSPITAL '}
{'Name': 610461753, 'Address__c': 'PO BOX 638704', 'City__c': 'CINCINNATI', 'State__c': 'OH', 'Zip__c': 452638704, 'Provider_Name__c': 'DEACONESS HENDERSON HOSPII '}
{'Name': 610461753, 'Address__c': 'PO BOX 638704', 'City__c': 'CINCINNATI', 'State__c': 'OH', 'Zip__c': 452638704, 'Provider_Name__c': 'DEACONESS HENDERSON HOSPII '}
{'Name': 610461753, 'Address__c': 'PO BOX 638704', 'City__c': 'CINCINNATI', 'State__c': 'OH', 'Zip__c': 452638704, 'Provider_Name__c': 'DEACONESS HENDERSON HOSPII '}
{'Name': 610461753, 'Address__c': 'PO BOX 638704', 'City__c': 'CINCINNATI', 'State__c': 'OH', 'Zip__c': 452638704, 'Provider_Name__c': 'DEACONESS HENDERSON HO

In [72]:
update_lines_object = upsert(map_lines_table, keys)


{'DOS__c': '2024-09-26', 'Rev_Code__c': 320, 'Billed_Amount__c': 987.0, 'Mod__c': None, 'Mod_2nd__c': None, 'Units__c': 1, 'MAR__c': 156.72, 'HCPCS_CPT_Code__c': 'a0b4W00000y1IFrQAM', 'Line_ID__c': 1, 'Exp_Code_2nd__c': None}
Error during upsert: Resource Line_Items__c Not Found. Response content: [{'errorCode': 'NOT_FOUND', 'message': 'Provided external ID field does not exist or is not accessible: RHP_ID__c'}]
{'DOS__c': '2024-10-17', 'Rev_Code__c': 510, 'Billed_Amount__c': 65.0, 'Mod__c': None, 'Mod_2nd__c': None, 'Units__c': 1, 'MAR__c': 65.0, 'HCPCS_CPT_Code__c': 'a0b4W00000y1JRaQAM', 'Line_ID__c': 1, 'Exp_Code_2nd__c': None}
Error during upsert: Resource Line_Items__c Not Found. Response content: [{'errorCode': 'NOT_FOUND', 'message': 'Provided external ID field does not exist or is not accessible: RHP_ID__c'}]
{'DOS__c': '2024-10-22', 'Rev_Code__c': 278, 'Billed_Amount__c': 3.0, 'Mod__c': None, 'Mod_2nd__c': None, 'Units__c': 1, 'MAR__c': 3.0, 'HCPCS_CPT_Code__c': None, 'Line_ID

KeyboardInterrupt: 

In [73]:
table_store['Line_Items__c'].head()

,Id,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastViewedDate,LastReferencedDate,DOS__c,Rev_Code__c,Line_Number__c,Billed_Amount__c,RHP_ID__c,Mod__c,Mod_2nd__c,Units__c,MAR__c,Allowed__c,Client__c,Reconsideration__c,Savings__c,Reductions__c,Bill_Type__c,Exp_Desc__c,HCPCS_CPT_Code__c,Additional_Payment__c,Line_ID__c,Pricing_Method__c,Provider_TIN_Claim__c,Jurisdiction_Claim__c,Pricing_Trigger__c,Exp_Code__c,Exp_Code_2nd__c,MO_QPA_Allowed__c,Exp_Desc_2nd__c,MO_QPA_Reductions__c,MO_Fac_QPA_Reductions__c,MO_Fac_QPA_Allowed__c,MO_Anes_QPA__c,QPA_Allowed__c,MO_Anes_QPA_Reductions__c,QPA_Reductions__c,Unique_Line_Id__c,Code_Edit__c,NDC_Number__c,Target_Amount__c
0,a0YUu000003hcTRMAY,False,L2283388,2024-04-20T11:23:21.000+0000,0054W00000ASrc5QAD,2024-04-20T19:22:43.000+0000,0054W00000DMWskQAH,2024-04-20T19:22:43.000+0000,None,None,2024-03-05,None,None,0.00,a06Uu000004eYf3IAE,None,None,1.0,0.00,0.00,a048a00000pNb0V,None,NaN,0.0,None,DIR Adjustment per PPO agreement or direct con...,a0b4W00000y1JE7QAM,None,3.0,SHELBY CO,a0F4W00000W1JNXUA3,a024W00000HYAjj,False,DIR,None,0.0,None,0.00,0.0,0.00,0.0,0.0,0.00,0.00,None,NaN,None,NaN
1,a0YUu000003hcTSMAY,False,L2283389,2024-04-20T11:23:21.000+0000,0054W00000ASrc5QAD,2024-04-20T19:22:43.000+0000,0054W00000DMWskQAH,2024-04-20T19:22:43.000+0000,None,None,2024-03-05,None,None,0.00,a06Uu000004eYf3IAE,None,None,1.0,0.00,0.00,a048a00000pNb0V,None,NaN,0.0,None,DIR Adjustment per PPO agreement or direct con...,a0b4W00000y1JDhQAM,None,4.0,SHELBY CO,a0F4W00000W1JNXUA3,a024W00000HYAjj,False,DIR,None,0.0,None,0.00,0.0,0.00,0.0,0.0,0.00,0.00,None,NaN,None,NaN
2,a0YUu000003hcTTMAY,False,L2283390,2024-04-20T11:23:21.000+0000,0054W00000ASrc5QAD,2024-04-20T11:23:23.000+0000,0054W00000ASrc5QAD,2024-04-20T11:23:23.000+0000,None,None,2023-12-07,None,None,134.16,a06Uu000004eYgbIAE,None,None,4.0,134.16,101.96,a048a00000pMtyg,None,24.0,32.2,None,FMP Reimbursement per fair market pricing benc...,a0b4W00000y1IZ6QAM,None,1.0,WORK COMP,a0F8a00001RfgbTEAR,a024W00000HYAjF,False,FMP,None,62.8,None,71.36,53.4,80.76,0.0,NaN,134.16,134.16,None,NaN,None,NaN
3,a0YUu000003hcTUMAY,False,L2283391,2024-04-20T11:23:21.000+0000,0054W00000ASrc5QAD,2024-04-20T11:23:23.000+0000,0054W00000ASrc5QAD,2024-04-20T11:23:23.000+0000,None,None,2024-03-13,None,None,134.16,a06Uu000004eYgcIAE,None,None,4.0,134.16,101.96,a048a00000pMtyg,None,24.0,32.2,None,FMP Reimbursement per fair market pricing benc...,a0b4W00000y1IZ6QAM,None,1.0,WORK COMP,a0F8a00001RfgeDEAR,a024W00000HYAjF,False,FMP,None,62.8,None,71.36,53.4,80.76,0.0,NaN,134.16,134.16,None,NaN,None,NaN
4,a0YUu000003hcTVMAY,False,L2283392,2024-04-20T11:23:21.000+0000,0054W00000ASrc5QAD,2024-04-20T11:23:23.000+0000,0054W00000ASrc5QAD,2024-04-20T11:23:23.000+0000,None,None,2024-03-13,None,None,0.00,a06Uu000004eYgcIAE,None,None,1.0,0.00,0.00,a048a00000pMtyg,None,NaN,0.0,None,FMP Reimbursement per fair market pricing benc...,a0b4W00000y1JlYQAU,None,2.0,WORK COMP,a0F8a00001RfgeDEAR,a024W00000HYAjF,False,FMP,None,0.0,None,0.00,0.0,0.00,0.0,NaN,0.00,0.00,None,NaN,None,NaN


In [ ]:
{'DOS__c': '2024-09-03', 'Rev_Code__c': 636, 'Billed_Amount__c': 26.54, 'Mod__c': None, 'Mod_2nd__c': None, 'Units__c': 100, 'MAR__c': 19.91, 'HCPCS_CPT_Code__c': 'a0bVF000000Sec1YAC', 'Line_ID__c': 6, 'Exp_Code_2nd__c': None}


In [ ]:
#### verifying upserts

Provider_data_pull = table_store['Provider_TIN__c']
Account_data_pull = table_store['Account']
Claim_data_pull = table_store['Claims__c']
Lines_data_pull = table_store['Line_Items__c']


provider_table_repulled = updated_data_pull('Provider_TIN__c', needed_objects)
claims_table_repulled = updated_data_pull('Claims__c', needed_objects)


print(Claim_data_pull.shape)




##providers pre upsert shape: 10823, post upsert shape: 10853
##claims pre upload shape: 12931, post upsert shape:

(12931, 99)


In [77]:
provider_table_repulled.shape



(12931, 99)

In [ ]:

unique_providers = list(map(str, set(Source_data['TIN'])))

Provider_data_pull[Provider_data_pull['Name'].isin(unique_providers)].shape
len(set(Source_data['TIN']))

59

In [ ]:
##### 29 providers on sandbox, 59 unique providers 

In [56]:
Provider_data_pull.head()

,Id,OwnerId,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastActivityDate,LastViewedDate,LastReferencedDate,Special_Repricing__c,Contact__c,Phone__c,Fax__c,Email__c,Notes__c,Unique_TIN__c,Address__c,City__c,State__c,Zip__c,Shelby_Co_Contract__c,City_of_Covington_Contract__c,Provider_Name__c,Avg_WC_Savings__c
0,a0F4W00000W1GxyUAF,0054W00000ASrc5QAD,False,000000002,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,2024-12-06T15:27:38.000+0000,2024-12-06T15:27:38.000+0000,False,None,None,None,None,None,000000002,PO Box 362,Royal Oak,MI,480680000,None,None,Provider Miscellaneous,0.0
1,a0F4W00000W1GxzUAF,0054W00000ASrc5QAD,False,000000009,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,None,None,False,None,None,None,None,None,000000009,[Replaced Address] 12288,COLUMBUS,GA,319179998,None,None,VISION EMPLOYEE CLAIMS,0.0
2,a0F4W00000W1Gy0UAF,0054W00000ASrc5QAD,False,00000002,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,None,None,False,None,None,None,None,None,00000002,None,None,None,None,None,None,MISCELLANEOUS PROVIDER,0.0
3,a0F4W00000W1Gy1UAF,0054W00000ASrc5QAD,False,000000021,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,None,None,False,None,None,None,None,None,000000021,[Replaced Address] 12288,COLUMBUS,GA,319179998,None,None,MEDICAL CLAIM,0.0
4,a0F4W00000W1Gy2UAF,0054W00000ASrc5QAD,False,000000022,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,None,None,False,None,None,None,None,None,000000022,[Replaced Address] 12288,COLUMBUS,GA,319179998,None,None,DENTAL CLAIMS,0.0
